### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.bootstrapper as bs
import fitter.fitter as fit
import fitter.special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [35]:
from importlib import reload
reload(dl)
reload(sf)
reload(fit)
reload(bs)

<module 'fitter.bootstrapper' from '../fitter/bootstrapper.py'>

### Specify fit parameters

In [29]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        
        # semi-nnlo corrections
        'include_alpha_s' : True,
        'include_log' : True, 
        'include_log2' : True,
        'include_sunset' : True,
        
        # nnnlo corrections
        'include_latt_n3lo' : True,
    }, 
    'fit_type' : 'xpt-ratio', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; experimental: 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'use_bijnens_central_value' : False,
    
    'use_prior' : True,
    'bias_correct' : True,
    'include_su2_isospin_corrrection' : False,
    'abbrs' : [u'a06m310L', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_results' : False,
    'save_pickles' : False,
    'output_name' : 'fit_results',
}

In [30]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], order=p_dict['order']['fit'], F2=p_dict['F2'], 
                  include_log=p_dict['order']['include_log'], include_log2=p_dict['order']['include_log2'], 
                  include_sunset=p_dict['order']['include_sunset'], include_alpha_s=p_dict['order']['include_alpha_s'], 
                  include_latt_n3lo=p_dict['order']['include_latt_n3lo'], include_FV=(p_dict['order']['vol'] > 6), 
                  use_bijnens_central_value=p_dict['use_bijnens_central_value']
             )
    
# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, phys_point_data, prior=prior, **p_dict)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print(bootstrapper)
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info(),
                              output_name=p_dict['output_name'], save_pickles=p_dict['save_pickles'])

    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

/mnt/Google/Documents/School Work/Research/fk-fpi/priors/xpt-ratio.csv
Using default prior.
Making fits...
Done!
100% complete
Time (s):  22.19511914253235
Compiling results...

Model: xpt-ratio_FKFpi_nnlo_log_logSq_sunset_alphaS_a4

Fitting to nnlo 
 with volume corrections O(0) 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections: False):

F_K / F_pi = 1.205(16) [1.1959(20)]   (delta_su2 = -0.00450(84))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      A_a 0                -20 (18)            1            0.00000 (22)
          1                 0 (100)        L_3 0             0.0013 (27)
     A_aa 0                 65 (34)            1             0.0000 (31)
          1                0 (1000)        L_4 0           -0.00042 (31)
      A_k 0               0.2 (2.8)            1             0.0000 (50)
          1               0.0 (5.0)        L_5 0       

In [34]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5,
    'L' : np.inf, 
    'a/w0' : 0,
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_a' : 0.1,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


bootstrapper.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

NameError: name 'bootstrapper' is not defined